In [1]:
#Last update by Moe at 21_Dec_2020

In [2]:
#flagged for one-hot encoding: DOMAIN; INCOME; GENDER; WEALTH; WEALTH2; GEOCODE2

In [3]:
#NOTE: RFA's subject to one-hot encoding but with caveat of not being optimal

In [4]:
#CHILD03 	CHILD07 	CHILD12 	CHILD18 	NUMCHLD

In [5]:
#ADATE_2 	ADATE_3 	ADATE_4 	ADATE_5 	ADATE_6 	ADATE_7 	ADATE_8 	ADATE_9 	ADATE_10 	ADATE_11 	ADATE_12 	ADATE_13 	ADATE_14 	ADATE_15 	ADATE_16 	ADATE_17 	ADATE_18 	ADATE_19 	ADATE_20 	ADATE_21 	ADATE_22 	ADATE_23 	ADATE_24 	RFA_2 	RFA_3 	RFA_4 	RFA_5 	RFA_6 	RFA_7 	RFA_8 	RFA_9 	RFA_10 	RFA_11 	RFA_12 	RFA_13 	RFA_14 	RFA_15 	RFA_16 	RFA_17 	RFA_18 	RFA_19 	RFA_20 	RFA_21 	RFA_22 	RFA_23 	RFA_24

# Group 

###  Create a customer segmentation 

**Group:**
- Joao Lucas, m2020
- Luis de Almeida, m2020
- Mohamed Elbawab, m20201102

In [6]:
#Loading libraries
import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np

import scipy as sp
from scipy.stats import spearmanr

import seaborn as sns

import math

from datetime import datetime
import calendar

import matplotlib
import matplotlib.pyplot as plt

import os

import sklearn
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import LeaveOneOut
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score # New
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedKFold

import mglearn

from itertools import product

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()

In [7]:
from sklearn.cluster import KMeans

In [8]:
from sklearn.impute import KNNImputer

In [9]:
data = pd.read_csv('donors.csv')
print(data.shape)
pd.options.display.max_columns = 476
data

(95412, 476)


,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,,,,XXXX,T2,,,,,,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,2017-06-01,2016-06-01,2016-04-01,2016-04-01,2016-03-01,

In [10]:
list(data.columns)

['Unnamed: 0',
 'ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'INCOME',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'E

In [11]:
data.describe(include="all")

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
count,95412.000000,95412,95412,95412.000000,95412,95412,95412,95412,71529,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,12386.000000,74126.000000,95412,50680.000000,95412.000000,42558.000000,42558.000000,42558.000000,42498.000000,42558.000000,42558.000000,42558.000000,42558.000000,42558.000000,42558.000000,42558.000000,42558.000000,42558.000000,42558.000000,95412,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412,95412,95412,51589.000000,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,

In [12]:
data.duplicated().sum()

0

In [13]:
null_values = pd.DataFrame(round((data.isna().sum()/len(data))*100,2))
null_values.rename(columns={0:'% of Null Values'}, inplace=True)
pd.options.display.max_rows = 476
null_values

,% of Null Values
Unnamed: 0,0.00
ODATEDW,0.00
OSOURCE,0.00
TCODE,0.00
STATE,0.00
ZIP,0.00
MAILCODE,0.00
PVASTATE,0.00
DOB,25.03
NOEXCH,0.00


In [14]:
prov_data = data.copy() 

In [15]:
prov_data["ZIP"] = prov_data["ZIP"].str.replace("-", "")
prov_data["ZIP"] = prov_data["ZIP"].astype(int) 

In [16]:
prov_data = prov_data.drop(prov_data[prov_data["MAILCODE"]=="B"].index)

In [17]:
prov_data = prov_data.drop(columns={"Unnamed: 0","MAILCODE","TCODE","NOEXCH","PVASTATE","MDMAUD","SOLP3","SOLIH","GEOCODE"})

In [18]:
prov_data["ODATEDW"] = pd.to_datetime(prov_data["ODATEDW"])

In [19]:
prov_data["DOB"] =  pd.to_datetime(prov_data["DOB"])

In [20]:
prov_data["RECINHSE"].replace("X",1,inplace=True)
prov_data["RECINHSE"].replace(" ",0,inplace=True)

In [21]:
prov_data["RECP3"].replace("X",1,inplace=True)
prov_data["RECP3"].replace(" ",0,inplace=True)

In [22]:
prov_data["RECPGVG"].replace("X",1,inplace=True)
prov_data["RECPGVG"].replace(" ",0,inplace=True)

In [23]:
prov_data["RECSWEEP"].replace("X",1,inplace=True)
prov_data["RECSWEEP"].replace(" ",0,inplace=True)

In [24]:
prov_data["DOMAIN"].replace(" ",np.nan,inplace=True)

In [25]:
prov_data["HOMEOWNR"].replace(" ",0,inplace=True)
prov_data["HOMEOWNR"].replace("H",1,inplace=True)
prov_data["HOMEOWNR"].replace("U",1,inplace=True)#acts as mode filling
prov_data["HOMEOWNR"].astype(int)

0        0
1        1
2        1
3        1
4        1
        ..
95407    0
95408    1
95409    0
95410    1
95411    1
Name: HOMEOWNR, Length: 94013, dtype: int32

In [26]:
prov_data['NUMCHLD'].fillna(0,inplace=True)
prov_data['NUMCHLD'].astype(int)

0        0
1        1
2        0
3        0
4        1
        ..
95407    0
95408    1
95409    0
95410    0
95411    0
Name: NUMCHLD, Length: 94013, dtype: int32

In [27]:
prov_data["DATASRCE"].replace(" ",np.nan,inplace=True)

In [28]:
prov_data["MAJOR"].replace(" ",0,inplace=True)
prov_data["MAJOR"].replace("X",1,inplace=True)

In [29]:
prov_data["COLLECT1"].replace(" ",0,inplace=True)
prov_data["COLLECT1"].replace("Y",1,inplace=True)
prov_data["VETERANS"].replace(" ",0,inplace=True)
prov_data["VETERANS"].replace("Y",1,inplace=True)
prov_data["BIBLE"].replace(" ",0,inplace=True)
prov_data["BIBLE"].replace("Y",1,inplace=True)
prov_data["CATLG"].replace(" ",0,inplace=True)
prov_data["CATLG"].replace("Y",1,inplace=True)
prov_data["HOMEE"].replace(" ",0,inplace=True)
prov_data["HOMEE"].replace("Y",1,inplace=True)
prov_data["PETS"].replace(" ",0,inplace=True)
prov_data["PETS"].replace("Y",1,inplace=True)
prov_data["CDPLAY"].replace(" ",0,inplace=True)
prov_data["CDPLAY"].replace("Y",1,inplace=True)
prov_data["STEREO"].replace(" ",0,inplace=True)
prov_data["STEREO"].replace("Y",1,inplace=True)
prov_data["PCOWNERS"].replace(" ",0,inplace=True)
prov_data["PCOWNERS"].replace("Y",1,inplace=True)
prov_data["PHOTO"].replace(" ",0,inplace=True)
prov_data["PHOTO"].replace("Y",1,inplace=True)
prov_data["CRAFTS"].replace(" ",0,inplace=True)
prov_data["CRAFTS"].replace("Y",1,inplace=True)
prov_data["FISHER"].replace(" ",0,inplace=True)
prov_data["FISHER"].replace("Y",1,inplace=True)
prov_data["GARDENIN"].replace(" ",0,inplace=True)
prov_data["GARDENIN"].replace("Y",1,inplace=True)
prov_data["BOATS"].replace(" ",0,inplace=True)
prov_data["BOATS"].replace("Y",1,inplace=True)
prov_data["WALKER"].replace(" ",0,inplace=True)
prov_data["WALKER"].replace("Y",1,inplace=True)
prov_data["KIDSTUFF"].replace(" ",0,inplace=True)
prov_data["KIDSTUFF"].replace("Y",1,inplace=True)
prov_data["CARDS"].replace(" ",0,inplace=True)
prov_data["CARDS"].replace("Y",1,inplace=True)
prov_data["PLATES"].replace(" ",0,inplace=True)
prov_data["PLATES"].replace("Y",1,inplace=True)

In [30]:
prov_data["LIFESRC"].replace(" ",np.nan,inplace=True)

In [31]:
prov_data["PEPSTRFL"].replace(" ",0,inplace=True)
prov_data["PEPSTRFL"].replace("X",1,inplace=True)

In [32]:
prov_data.loc[:,"MALEMILI":"FEDGOV"] = prov_data.loc[:,"MALEMILI":"FEDGOV"]/100 

In [33]:
prov_data.loc[:,"POP90C1":"ETH16"] = prov_data.loc[:,"POP90C1":"ETH16"]/100

In [34]:
prov_data.loc[:,"AGE907":"MARR4"] = prov_data.loc[:,"AGE907":"MARR4"]/100   

In [35]:
prov_data.loc[:,"DW1":"DW9"] = prov_data.loc[:,"DW1":"DW9"]/100 

In [36]:
prov_data.loc[:,"HU1":"HUR2"] = prov_data.loc[:,"HU1":"HUR2"]/100 

In [37]:
prov_data.loc[:,"HUPA1":"RP4"] = prov_data.loc[:,"HUPA1":"RP4"]/100 

In [38]:
prov_data.loc[:,"IC6":"PEC2"] = prov_data.loc[:,"IC6":"PEC2"]/100 

In [39]:
prov_data.loc[:,"TPE12":"OEDC7"] = prov_data.loc[:,"TPE12":"OEDC7"]/100 

In [40]:
prov_data.loc[:,"EC2":"HC21"] = prov_data.loc[:,"EC2":"HC21"]/100 

In [41]:
prov_data.loc[:,"AC1":"AC2"] = prov_data.loc[:,"AC1":"AC2"]/100 

In [42]:
prov_data["ADATE_2"] = pd.to_datetime(prov_data["ADATE_2"])
prov_data["ADATE_3"] = pd.to_datetime(prov_data["ADATE_3"])
prov_data["ADATE_4"] = pd.to_datetime(prov_data["ADATE_4"])
prov_data["ADATE_5"] = pd.to_datetime(prov_data["ADATE_5"])
prov_data["ADATE_6"] = pd.to_datetime(prov_data["ADATE_6"])
prov_data["ADATE_7"] = pd.to_datetime(prov_data["ADATE_7"])
prov_data["ADATE_8"] = pd.to_datetime(prov_data["ADATE_8"])
prov_data["ADATE_9"] = pd.to_datetime(prov_data["ADATE_9"])
prov_data["ADATE_10"] = pd.to_datetime(prov_data["ADATE_10"])
prov_data["ADATE_11"] = pd.to_datetime(prov_data["ADATE_11"])
prov_data["ADATE_12"] = pd.to_datetime(prov_data["ADATE_12"])
prov_data["ADATE_13"] = pd.to_datetime(prov_data["ADATE_13"])
prov_data["ADATE_14"] = pd.to_datetime(prov_data["ADATE_14"])
prov_data["ADATE_15"] = pd.to_datetime(prov_data["ADATE_15"])
prov_data["ADATE_16"] = pd.to_datetime(prov_data["ADATE_16"])
prov_data["ADATE_17"] = pd.to_datetime(prov_data["ADATE_17"])
prov_data["ADATE_18"] = pd.to_datetime(prov_data["ADATE_18"])
prov_data["ADATE_19"] = pd.to_datetime(prov_data["ADATE_19"])
prov_data["ADATE_20"] = pd.to_datetime(prov_data["ADATE_20"])
prov_data["ADATE_21"] = pd.to_datetime(prov_data["ADATE_21"])
prov_data["ADATE_22"] = pd.to_datetime(prov_data["ADATE_22"])
prov_data["ADATE_23"] = pd.to_datetime(prov_data["ADATE_23"])
prov_data["ADATE_24"] = pd.to_datetime(prov_data["ADATE_24"])

In [43]:
prov_data["MAXADATE"] = pd.to_datetime(prov_data["MAXADATE"])

In [44]:
prov_data["RDATE_3"] = pd.to_datetime(prov_data["RDATE_3"])
prov_data["RDATE_4"] = pd.to_datetime(prov_data["RDATE_4"])
prov_data["RDATE_5"] = pd.to_datetime(prov_data["RDATE_5"])
prov_data["RDATE_6"] = pd.to_datetime(prov_data["RDATE_6"])
prov_data["RDATE_7"] = pd.to_datetime(prov_data["RDATE_7"])
prov_data["RDATE_8"] = pd.to_datetime(prov_data["RDATE_8"])
prov_data["RDATE_9"] = pd.to_datetime(prov_data["RDATE_9"])
prov_data["RDATE_10"] = pd.to_datetime(prov_data["RDATE_10"])
prov_data["RDATE_11"] = pd.to_datetime(prov_data["RDATE_11"])
prov_data["RDATE_12"] = pd.to_datetime(prov_data["RDATE_12"])
prov_data["RDATE_13"] = pd.to_datetime(prov_data["RDATE_13"])
prov_data["RDATE_14"] = pd.to_datetime(prov_data["RDATE_14"])
prov_data["RDATE_15"] = pd.to_datetime(prov_data["RDATE_15"])
prov_data["RDATE_16"] = pd.to_datetime(prov_data["RDATE_16"])
prov_data["RDATE_17"] = pd.to_datetime(prov_data["RDATE_17"])
prov_data["RDATE_18"] = pd.to_datetime(prov_data["RDATE_18"])
prov_data["RDATE_19"] = pd.to_datetime(prov_data["RDATE_19"])
prov_data["RDATE_20"] = pd.to_datetime(prov_data["RDATE_20"])
prov_data["RDATE_21"] = pd.to_datetime(prov_data["RDATE_21"])
prov_data["RDATE_22"] = pd.to_datetime(prov_data["RDATE_22"])
prov_data["RDATE_23"] = pd.to_datetime(prov_data["RDATE_23"])
prov_data["RDATE_24"] = pd.to_datetime(prov_data["RDATE_24"])

In [45]:
prov_data["MINRDATE"] = pd.to_datetime(prov_data["MINRDATE"])

In [46]:
prov_data["MAXRDATE"] = pd.to_datetime(prov_data["MAXRDATE"])

In [47]:
prov_data["LASTDATE"] = pd.to_datetime(prov_data["LASTDATE"])
prov_data["FISTDATE"] = pd.to_datetime(prov_data["FISTDATE"])
prov_data["NEXTDATE"] = pd.to_datetime(prov_data["NEXTDATE"])

In [48]:
prov_data = prov_data.drop(columns=data.loc[:,"ADATE_2":"RFA_24"].columns)

In [49]:
prov_data = prov_data.drop(columns=data.loc[:,"RDATE_3":"RAMNT_24"].columns)

In [50]:
prov_data.reset_index(drop=True,inplace=True)

In [51]:
prov_data["LIFETIME_OF_DONOR_DATA"] = prov_data["LASTDATE"]-prov_data["ODATEDW"]

In [52]:
prov_data["LIFETIME_OF_DONOR_DATA"] = prov_data["LIFETIME_OF_DONOR_DATA"].dt.days

In [53]:
prov_data = prov_data.drop(columns="ODATEDW")

In [54]:
#prov_data = prov_data.set_index("CONTROLN")

In [55]:
prov_data

,OSOURCE,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,MAJOR,WEALTH2,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,LIFETIME_OF_DONOR_DATA
0,GRI,IL,61081,1957-12-01,0,0,0,0,T2,0,,,,,0.0,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.39,0.34,0.18,0.10,0.02,0.01,0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1,992,264,332,0.00,0.35,0.65,0.47,0.53,0.92,0.01,0.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.11,0.00,0.00,0.00,39,48,51,40,50,54,0.25,0.31,0.42,0.27,0.11,0.14,0.18,0.17,0.13,0.11,0.15,0.12,0.11,0.34,0.25,0.18,0.26,0.10,0.23,0.18,0.33,0.49,0.28,0.12,0.04,0.61,0.07,0.12,0.19,198,276,0.97,0.95,0.02,0.02,0.00,0.00,0.07,0.07,0.00,479,635,3,2,0.86,0.14,0.96,0.04,0.07,0.38,0.80,0.70,0.32,0.84,0.16,0.06,0.02,0.05,0.09,0.15,0.03,0.17,0.50,0.25,0.00,0.00,0.00,0.02,0.07,0.13,0.27,0.47,0.00,0.01,0.61,58,61,15,4,0.02,0.00,0.00,0.14,0.01,0.00,0.00,0.02,0.05,0.17,0.73,0.0,177.0,682.0,307,318,349,378,12883,0.13,0.23,0.23,0.23,0.15,0.01,0.00,0.00,0.01,0.04,0.25,0.24,0.26,0.17,0.02,0.00,0.00,0.02,0.28,0.04,0.51,0.01,0.46,0.54,0.03,0.88,0.08,0.00,0.00,0.0,0.0,0.00,0.00,0.04,0.01,0.13,14,16,0.02,0.45,0.56,0.64,0.50,0.64,0.44,0.62,0.53,0.99,0.00,0.00,0.09,0.03,0.08,0.13,0.09,0.00,0.03,0.09,0.03,0.15,0.19,0.05,0.04,0.03,0.00,0.03,0.41,0.01,0.00,0.07,0.13,0.06,0.05,0.00,0.04,0.09,0.04,0.01,0.03,0.10,0.02,0.01,0.07,0.78,0.02,0.00,120,0.16,0.10,0.39,0.21,0.08,0.04,0.03,0.05,0.20,0.03,0.19,0.04,0.00,0.00,0.00,0.18,0.39,0.00,0.34,0.23,0.18,0.16,0.01,0.04,0.00,0.23,0.00,0.00,0.05,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.0,0.03,0.74,0.88,0.08,0.00,0.04,0.96,0.77,0.19,0.13,0.31,0.05,0.14,0.14,0.31,0.54,0.46,0.00,0.00,0.90,0.00,0.10,0.00,0.0,0.00,0.33,0.65,0.40,0.99,0.99,6,2,0.10,0.07,27,2017-02-01,74,6,14,240.0,31,14,5.0,2012-08-01,12.0,2014-02-01,10.0,2015-12-01,2009-11-01,2010-03-01,4.0,7.741935,95515,0,L,4,E,X,X,X,C,2525
1,BOA,CA,91326,1972-02-01,0,0,0,0,S1,1,,,,M,

In [56]:
prov_data.dtypes

OSOURCE                           object
STATE                             object
ZIP                                int32
DOB                       datetime64[ns]
RECINHSE                           int64
RECP3                              int64
RECPGVG                            int64
RECSWEEP                           int64
DOMAIN                            object
HOMEOWNR                           int64
CHILD03                           object
CHILD07                           object
CHILD12                           object
CHILD18                           object
NUMCHLD                          float64
INCOME                           float64
GENDER                            object
WEALTH1                          float64
HIT                                int64
MBCRAFT                          float64
MBGARDEN                         float64
MBBOOKS                          float64
MBCOLECT                         float64
MAGFAML                          float64
MAGFEM          

In [57]:
prov_data["ADI"].unique().tolist()

[177.0,
 13.0,
 281.0,
 67.0,
 127.0,
 185.0,
 91.0,
 251.0,
 391.0,
 181.0,
 269.0,
 83.0,
 107.0,
 249.0,
 291.0,
 57.0,
 227.0,
 351.0,
 55.0,
 133.0,
 51.0,
 73.0,
 159.0,
 201.0,
 75.0,
 233.0,
 591.0,
 203.0,
 175.0,
 59.0,
 105.0,
 111.0,
 173.0,
 415.0,
 323.0,
 187.0,
 65.0,
 412.0,
 109.0,
 425.0,
 405.0,
 197.0,
 279.0,
 645.0,
 207.0,
 89.0,
 361.0,
 209.0,
 213.0,
 15.0,
 21.0,
 113.0,
 131.0,
 469.0,
 199.0,
 129.0,
 441.0,
 459.0,
 301.0,
 39.0,
 429.0,
 285.0,
 245.0,
 335.0,
 393.0,
 577.0,
 439.0,
 455.0,
 383.0,
 263.0,
 427.0,
 329.0,
 307.0,
 275.0,
 119.0,
 0.0,
 229.0,
 273.0,
 325.0,
 371.0,
 377.0,
 315.0,
 342.0,
 421.0,
 219.0,
 61.0,
 93.0,
 375.0,
 283.0,
 243.0,
 353.0,
 613.0,
 339.0,
 367.0,
 417.0,
 299.0,
 271.0,
 157.0,
 77.0,
 327.0,
 53.0,
 87.0,
 451.0,
 627.0,
 71.0,
 253.0,
 363.0,
 403.0,
 115.0,
 389.0,
 63.0,
 321.0,
 448.0,
 237.0,
 217.0,
 241.0,
 123.0,
 221.0,
 165.0,
 277.0,
 85.0,
 319.0,
 247.0,
 69.0,
 379.0,
 95.0,
 17.0,
 625.0,
 462

In [58]:
prov_data["MSA"].isna().sum()

121

In [59]:
prov_data = prov_data.drop(columns=data.loc[:,"CHILD03":"CHILD18"].columns) #dropped child03 to child18

In [60]:
metric_features = prov_data.loc[:,"POP901":"RP4"].columns.tolist()
metric_features_2 = ["NUMCHLD","HIT","MALEMILI","MALEVET","VIETVETS","WWIIVETS","LOCALGOV","STATEGOV","FEDGOV"]
metric_features_3 = prov_data.loc[:,"IC1":"AC2"].columns.tolist()
metric_features_4 = ["CARDPROM","NUMPROM","CARDPM12","NUMPRM12","RAMNTALL","NGIFTALL","CARDGIFT","MINRAMNT","MAXRAMNT","LASTGIFT","TIMELAG","AVGGIFT"]

In [61]:
metric_features.extend(metric_features_2)
metric_features.extend(metric_features_3)
metric_features.extend(metric_features_4)

In [62]:
non_metric_features = prov_data.columns.drop(metric_features).to_list()

In [63]:
# Filling the categorical values
prov_data_central = prov_data.copy()
prov_data_central.isna().sum()

OSOURCE                       0
STATE                         0
ZIP                           0
DOB                       23395
RECINHSE                      0
RECP3                         0
RECPGVG                       0
RECSWEEP                      0
DOMAIN                     2221
HOMEOWNR                      0
NUMCHLD                       0
INCOME                    20806
GENDER                        0
WEALTH1                   43895
HIT                           0
MBCRAFT                   51885
MBGARDEN                  51885
MBBOOKS                   51885
MBCOLECT                  51945
MAGFAML                   51885
MAGFEM                    51885
MAGMALE                   51885
PUBGARDN                  51885
PUBCULIN                  51885
PUBHLTH                   51885
PUBDOITY                  51885
PUBNEWFN                  51885
PUBPHOTO                  51885
PUBOPP                    51885
DATASRCE                  20801
MALEMILI                      0
MALEVET 

In [64]:
modes = prov_data_central[non_metric_features].mode().loc[0]
modes

OSOURCE                                   MBC
STATE                                      CA
ZIP                                     85351
DOB                       1968-01-01 00:00:00
RECINHSE                                    0
RECP3                                       0
RECPGVG                                     0
RECSWEEP                                    0
DOMAIN                                     R2
HOMEOWNR                                    1
INCOME                                      5
GENDER                                      F
WEALTH1                                     9
MBCRAFT                                     0
MBGARDEN                                    0
MBBOOKS                                     0
MBCOLECT                                    0
MAGFAML                                     0
MAGFEM                                      0
MAGMALE                                     0
PUBGARDN                                    0
PUBCULIN                          

In [65]:
prov_data_central.fillna(modes, inplace=True)
prov_data_central.isna().sum()  # checking how many NaNs we still have

OSOURCE                      0
STATE                        0
ZIP                          0
DOB                          0
RECINHSE                     0
RECP3                        0
RECPGVG                      0
RECSWEEP                     0
DOMAIN                       0
HOMEOWNR                     0
NUMCHLD                      0
INCOME                       0
GENDER                       0
WEALTH1                      0
HIT                          0
MBCRAFT                      0
MBGARDEN                     0
MBBOOKS                      0
MBCOLECT                     0
MAGFAML                      0
MAGFEM                       0
MAGMALE                      0
PUBGARDN                     0
PUBCULIN                     0
PUBHLTH                      0
PUBDOITY                     0
PUBNEWFN                     0
PUBPHOTO                     0
PUBOPP                       0
DATASRCE                     0
MALEMILI                     0
MALEVET                      0
VIETVETS

In [66]:
prov_data= prov_data_central.copy()

In [67]:
#KNN - For numerical varaibles
prov_data_neighbors = prov_data.copy()

# Seeing rows with NaNs
nans_index = prov_data_neighbors.isna().any(axis=1)
prov_data_neighbors[nans_index] # checking how many NaNs we still have

,OSOURCE,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,HOMEOWNR,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,MAJOR,WEALTH2,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,LIFETIME_OF_DONOR_DATA
10,AGR,KS,67335,1968-01-01,0,0,0,0,R3,1,0.0,2.0,F,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00,0.38,0.31,0.46,0.06,0.05,0.02,0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3356,922,1345,0.00,0.00,0.99,0.47,0.53,0.98,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.0,0.00,36,47,50,39,49,53,0.26,0.41,0.39,0.19,0.10,0.18,0.17,0.14,0.14,0.13,0.13,0.15,0.18,0.33,0.21,0.12,0.35,0.19,0.34,0.30,0.31,0.40,0.24,0.08,0.03,0.64,0.09,0.12,0.15,166,246,0.83,0.83,0.02,0.06,0.04,0.01,0.01,0.01,0.00,231,273,2,1,0.77,0.23,0.88,0.12,0.02,0.34,0.69,0.60,0.29,0.84,0.16,0.06,0.01,0.05,0.11,0.24,0.02,0.22,0.52,0.24,0.00,0.00,0.0,0.00,0.00,0.01,0.02,0.12,0.00,0.02,0.44,52,54,13,4,0.06,0.00,0.10,0.14,0.05,0.01,0.03,0.00,0.01,0.05,0.28,0.0,269.0,671.0,173,222,220,273,9045,0.43,0.24,0.15,0.11,0.06,0.00,0.00,0.00,0.00,0.25,0.30,0.19,0.16,0.09,0.00,0.00,0.00,0.01,0.41,0.08,0.32,0.17,0.34,0.66,0.08,0.70,0.19,0.00,0.00,0.00,0.0,0.00,0.06,0.05,0.01,0.10,17,22,0.06,0.50,0.59,0.68,0.52,0.66,0.48,0.67,0.54,0.68,0.65,0.06,0.05,0.08,0.01,0.06,0.14,0.00,0.01,0.13,0.05,0.18,0.19,0.04,0.07,0.05,0.02,0.05,0.37,0.03,0.03,0.01,0.13,0.05,0.04,0.03,0.01,0.06,0.05,0.05,0.04,0.06,0.06,0.02,0.09,0.72,0.04,0.01,120,0.12,0.17,0.38,0.20,0.05,0.05,0.02,0.01,0.27,0.02,0.21,0.05,0.00,0.00,0.00,0.17,0.38,0.00,0.31,0.21,0.46,0.04,0.00,0.06,0.00,0.10,0.00,0.00,0.04,0.00,0.01,0.00,0.0,0.00,0.01,0.01,0.0,0.00,0.77,0.98,0.01,0.00,0.01,0.89,0.54,0.17,0.09,0.47,0.00,0.04,0.08,0.23,0.31,0.69,0.00,0.00,0.76,0.06,0.11,0.00,0.0,0.08,0.97,0.02,0.76,0.99,0.90,8,2,0.05,0.08,6,2017-02-01,14,5,12,20.0,1,1,20.0,2016-02-01,20.0,2016-02-01,20.0,2016-02-01,2016-02-01,2015-04-01,NaN,20.0,106016,0,L,1,F,X,X,X,D,31
20,L21,MI,49247,1936-01-01,0,0,0,0,R2,1,0.0,2.0,M,3.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [68]:
# KNNImputer 
imputer = KNNImputer(n_neighbors=5, weights="uniform")
prov_data_neighbors[metric_features] = imputer.fit_transform(prov_data_neighbors[metric_features])

# See rows with NaNs imputed
prov_data_neighbors.loc[nans_index, metric_features]

,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,NUMCHLD,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT
10,3356.0,922.0,1345.0,0.00,0.00,0.99,0.47,0.53,0.98,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.0,0.00,36.0,47.0,50.0,39.0,49.0,53.0,0.26,0.41,0.39,0.19,0.10,0.18,0.17,0.14,0.14,0.13,0.13,0.15,0.18,0.33,0.21,0.12,0.35,0.19,0.34,0.30,0.31,0.40,0.24,0.08,0.03,0.64,0.09,0.12,0.15,166.0,246.0,0.83,0.83,0.02,0.06,0.04,0.01,0.01,0.01,0.00,231.0,273.0,2.0,1.0,0.77,0.23,0.88,0.12,0.02,0.34,0.69,0.60,0.29,0.84,0.16,0.06,0.01,0.05,0.11,0.24,0.02,0.22,0.52,0.24,0.00,0.00,0.0,0.00,0.00,0.01,0.02,0.12,0.00,0.02,0.44,52.0,54.0,13.0,4.0,0.06,0.00,0.10,0.14,0.05,0.01,0.03,0.00,0.01,0.05,0.28,0.0,0.0,0.00,0.38,0.31,0.46,0.06,0.05,0.02,173.0,222.0,220.0,273.0,9045.0,0.43,0.24,0.15,0.11,0.06,0.00,0.00,0.00,0.00,0.25,0.30,0.19,0.16,0.09,0.00,0.00,0.00,0.01,0.41,0.08,0.32,0.17,0.34,0.66,0.08,0.70,0.19,0.00,0.00,0.00,0.0,0.00,0.06,0.05,0.01,0.10,17.0,22.0,0.06,0.50,0.59,0.68,0.52,0.66,0.48,0.67,0.54,0.68,0.65,0.06,0.05,0.08,0.01,0.06,0.14,0.00,0.01,0.13,0.05,0.18,0.19,0.04,0.07,0.05,0.02,0.05,0.37,0.03,0.03,0.01,0.13,0.05,0.04,0.03,0.01,0.06,0.05,0.05,0.04,0.06,0.06,0.02,0.09,0.72,0.04,0.01,120.0,0.12,0.17,0.38,0.20,0.05,0.05,0.02,0.01,0.27,0.02,0.21,0.05,0.00,0.00,0.00,0.17,0.38,0.00,0.31,0.21,0.46,0.04,0.00,0.06,0.00,0.10,0.00,0.00,0.04,0.00,0.01,0.00,0.0,0.00,0.01,0.01,0.0,0.00,0.77,0.98,0.01,0.00,0.01,0.89,0.54,0.17,0.09,0.47,0.00,0.04,0.08,0.23,0.31,0.69,0.00,0.00,0.76,0.06,0.11,0.00,0.0,0.08,0.97,0.02,0.76,0.99,0.90,8.0,2.0,0.05,0.08,6.0,14.0,5.0,12.0,20.0,1.0,1.0,20.0,20.0,20.0,7.0,20.0
20,1112.0,327.0,406.0,0.00,0.00,0.99,0.49,0.51,0.99,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.0,0.00,30.0,40.0,44.0,33.0,44.0,47.0,0.30,0.37,0.38,0.25,0.14,0.21,0.23,0.15,0.13,0.08,0.06,0.16,0.16,0.29,0.23,0.16,0.20,0.07,0.18,0.18,0.35,0.48,0.28,0.12,0.03,0.65,0.10,0.05,0.19,193.0,274.0,0.68,0.67,0.02,0.10,0.08,0.04,0.00,0.00,0.00,423.0,460.0,3.0,3.0,0.83,0.17,0.91,0.09,0.64,0.43,0.81,0.66,0.32,0.91,0.09,0.11,0.02,0.08,0.08,0.15,0.02,0.24,0.61,0.14,0.00,0.01,0.0,0.00,0.00,0.02,0.11,0.37,0.00,0.02,0.50,57.0,58.0,15.0,4.0,0.08,0.02,0.21,0.05,0.05,0.05,0.01,0.02,0.08,0.39,0.89,0.0,0.0,0.00,0.28,0.36,0.20,0.09,0.03,0.01,301.0,330.0,338.0,372.0,12311.0,0.15,0.24,0.21,0.23,0.13,0.03,0.01,0.00,0.01,0.10,0.22,0.20,0.27,0.16,0.04,0.01,0.00,0.01,0.22,0.07,0.48,0.06,0.51,0.49,0.04,0.71,0.17,0.00

In [69]:
prov_data_neighbors.loc[nans_index]

,OSOURCE,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,HOMEOWNR,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,MAJOR,WEALTH2,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,LIFETIME_OF_DONOR_DATA
10,AGR,KS,67335,1968-01-01,0,0,0,0,R3,1,0.0,2.0,F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00,0.38,0.31,0.46,0.06,0.05,0.02,0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3356.0,922.0,1345.0,0.00,0.00,0.99,0.47,0.53,0.98,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.0,0.00,36.0,47.0,50.0,39.0,49.0,53.0,0.26,0.41,0.39,0.19,0.10,0.18,0.17,0.14,0.14,0.13,0.13,0.15,0.18,0.33,0.21,0.12,0.35,0.19,0.34,0.30,0.31,0.40,0.24,0.08,0.03,0.64,0.09,0.12,0.15,166.0,246.0,0.83,0.83,0.02,0.06,0.04,0.01,0.01,0.01,0.00,231.0,273.0,2.0,1.0,0.77,0.23,0.88,0.12,0.02,0.34,0.69,0.60,0.29,0.84,0.16,0.06,0.01,0.05,0.11,0.24,0.02,0.22,0.52,0.24,0.00,0.00,0.0,0.00,0.00,0.01,0.02,0.12,0.00,0.02,0.44,52.0,54.0,13.0,4.0,0.06,0.00,0.10,0.14,0.05,0.01,0.03,0.00,0.01,0.05,0.28,0.0,269.0,671.0,173.0,222.0,220.0,273.0,9045.0,0.43,0.24,0.15,0.11,0.06,0.00,0.00,0.00,0.00,0.25,0.30,0.19,0.16,0.09,0.00,0.00,0.00,0.01,0.41,0.08,0.32,0.17,0.34,0.66,0.08,0.70,0.19,0.00,0.00,0.00,0.0,0.00,0.06,0.05,0.01,0.10,17.0,22.0,0.06,0.50,0.59,0.68,0.52,0.66,0.48,0.67,0.54,0.68,0.65,0.06,0.05,0.08,0.01,0.06,0.14,0.00,0.01,0.13,0.05,0.18,0.19,0.04,0.07,0.05,0.02,0.05,0.37,0.03,0.03,0.01,0.13,0.05,0.04,0.03,0.01,0.06,0.05,0.05,0.04,0.06,0.06,0.02,0.09,0.72,0.04,0.01,120.0,0.12,0.17,0.38,0.20,0.05,0.05,0.02,0.01,0.27,0.02,0.21,0.05,0.00,0.00,0.00,0.17,0.38,0.00,0.31,0.21,0.46,0.04,0.00,0.06,0.00,0.10,0.00,0.00,0.04,0.00,0.01,0.00,0.0,0.00,0.01,0.01,0.0,0.00,0.77,0.98,0.01,0.00,0.01,0.89,0.54,0.17,0.09,0.47,0.00,0.04,0.08,0.23,0.31,0.69,0.00,0.00,0.76,0.06,0.11,0.00,0.0,0.08,0.97,0.02,0.76,0.99,0.90,8.0,2.0,0.05,0.08,6.0,2017-02-01,14.0,5.0,12.0,20.0,1.0,1.0,20.0,2016-02-01,20.0,2016-02-01,20.0,2016-02-01,2016-02-01,2015-04-01,7.0,20.0,106016,0,L,1,F,X,X,X,D,31
20,L21,MI,49247,193

In [70]:
prov_data= prov_data_neighbors.copy()

In [71]:
prov_data

,OSOURCE,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,DOMAIN,HOMEOWNR,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,MAJOR,WEALTH2,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,LIFETIME_OF_DONOR_DATA
0,GRI,IL,61081,1957-12-01,0,0,0,0,T2,0,0.0,5.0,F,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.00,0.39,0.34,0.18,0.10,0.02,0.01,0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,992.0,264.0,332.0,0.00,0.35,0.65,0.47,0.53,0.92,0.01,0.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.11,0.00,0.00,0.00,39.0,48.0,51.0,40.0,50.0,54.0,0.25,0.31,0.42,0.27,0.11,0.14,0.18,0.17,0.13,0.11,0.15,0.12,0.11,0.34,0.25,0.18,0.26,0.10,0.23,0.18,0.33,0.49,0.28,0.12,0.04,0.61,0.07,0.12,0.19,198.0,276.0,0.97,0.95,0.02,0.02,0.00,0.00,0.07,0.07,0.00,479.0,635.0,3.0,2.0,0.86,0.14,0.96,0.04,0.07,0.38,0.80,0.70,0.32,0.84,0.16,0.06,0.02,0.05,0.09,0.15,0.03,0.17,0.50,0.25,0.00,0.00,0.00,0.02,0.07,0.13,0.27,0.47,0.00,0.01,0.61,58.0,61.0,15.0,4.0,0.02,0.00,0.00,0.14,0.01,0.00,0.00,0.02,0.05,0.17,0.73,0.0,177.0,682.0,307.0,318.0,349.0,378.0,12883.0,0.13,0.23,0.23,0.23,0.15,0.01,0.00,0.00,0.01,0.04,0.25,0.24,0.26,0.17,0.02,0.00,0.00,0.02,0.28,0.04,0.51,0.01,0.46,0.54,0.03,0.88,0.08,0.00,0.00,0.0,0.0,0.00,0.00,0.04,0.01,0.13,14.0,16.0,0.02,0.45,0.56,0.64,0.50,0.64,0.44,0.62,0.53,0.99,0.00,0.00,0.09,0.03,0.08,0.13,0.09,0.00,0.03,0.09,0.03,0.15,0.19,0.05,0.04,0.03,0.00,0.03,0.41,0.01,0.00,0.07,0.13,0.06,0.05,0.00,0.04,0.09,0.04,0.01,0.03,0.10,0.02,0.01,0.07,0.78,0.02,0.00,120.0,0.16,0.10,0.39,0.21,0.08,0.04,0.03,0.05,0.20,0.03,0.19,0.04,0.00,0.00,0.00,0.18,0.39,0.00,0.34,0.23,0.18,0.16,0.01,0.04,0.00,0.23,0.00,0.00,0.05,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.0,0.03,0.74,0.88,0.08,0.00,0.04,0.96,0.77,0.19,0.13,0.31,0.05,0.14,0.14,0.31,0.54,0.46,0.00,0.00,0.90,0.00,0.10,0.00,0.0,0.00,0.33,0.65,0.40,0.99,0.99,6.0,2.0,0.10,0.07,27.0,2017-02-01,74.0,6.0,14.0,240.0,31.0,14.0,5.0,2012-08-01,12.0,2014-02-01,10.0,2015-12-01,2009-11-01,2010-03-01,4.0,7.741935,95515,0,L,4,E,X,X,X,C,2525
1,BOA,CA,913

In [72]:
#prov_data = prov_data.drop(columns=data.loc[:,"CHILD03":"CHILD18"].columns) #dropped child03 to child18

In [73]:
#prov_data

###  Data Skewness & Normalization 


In [74]:
#Will be done later
prov_data_normalized_metric = prov_data[metric_features].copy()
prov_data_normalized_non_metric = prov_data[non_metric_features].copy()

###  Determining Hyper Parameters - Elbow MEthod 

In [76]:
sse = {}
for k in range(1, 373):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(prov_data_normalized_metric)
    sse[k] = kmeans.inertia_ # SSE to closest cluster centroid
plt.title('The Elbow Method')
plt.xlabel('k')
plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

KeyboardInterrupt: 